In [1]:
import sys
from pathlib import Path
import warnings

import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

import sys
# Custom library paths
sys.path.extend(['../', './scr'])

from scr.utils import set_seed
from scr.utils import read_words
from pathlib import Path
import random
from collections import Counter, defaultdict
import pickle
from tqdm import tqdm
from torch.utils.data import Dataset
from scr.feature_engineering import \
    calculate_char_frequencies, calculate_word_frequencies
from scr.utils import read_words, save_words_to_file

import pickle
from pathlib import Path
from scr.dataset import *
from scr.game import *
import gc

set_seed(42)

import torch
import torch.nn as nn
from pathlib import Path
import random

from scr.utils import print_scenarios
torch.set_float32_matmul_precision('medium')

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Read and Shuffle Word List
word_list = read_words('data/words_250000_train.txt') # , limit=10000)
# word_list = read_words('data/250k.txt', limit=10000)
random.shuffle(word_list)

# base_dataset_dir = Path('dataset/pkl')

In [2]:
from pathlib import Path

NUM_STRATIFIED_SAMPLES = 2500

# Define the base directory
base_dataset_dir = Path(f'./dataset/{NUM_STRATIFIED_SAMPLES}')
# base_dataset_dir = Path(f"/media/sayem/510B93E12554BBD1/dataset/{NUM_STRATIFIED_SAMPLES}")

# Ensuring the base directory and 'pkl' subdirectory exist
base_dataset_dir.mkdir(parents=True, exist_ok=True)
pkls_dir = base_dataset_dir / 'pkl'
pkls_dir.mkdir(parents=True, exist_ok=True)

# Paths to the words files
train_words_file_path = base_dataset_dir / 'train_words.txt'
val_words_file_path = base_dataset_dir / 'val_words.txt'

# Read the words from the files
try:
    train_words = read_words(train_words_file_path)
    print(f"Loaded {len(train_words)} train words from {train_words_file_path}")
except FileNotFoundError:
    print(f"File not found: {train_words_file_path}")

try:
    val_words = read_words(val_words_file_path)
    print(f"Loaded {len(val_words)} validation words from {val_words_file_path}")
except FileNotFoundError:
    print(f"File not found: {val_words_file_path}")

Loaded 159110 train words from dataset/2500/train_words.txt
Loaded 34095 validation words from dataset/2500/val_words.txt


In [3]:
base_dataset_dir

PosixPath('dataset/2500')

In [4]:
# # For inference
from scr.feature_engineering import *

word_frequencies = calculate_word_frequencies(word_list)
char_frequency = calculate_char_frequencies(word_list)
max_word_length = max(len(word) for word in word_list)

##### Model Building

In [5]:
from scr.simple_model import SimpleLSTM
from scr.base_model import BaseModel
from pathlib import Path
import torch
from scr.feature_engineering import *

# Instantiate and test the model
config = {
    'embedding_dim': 200,
    'hidden_dim': 256,
    'num_layers': 2,
    'vocab_size': 27,
    'max_word_length': max_word_length,
    'input_feature_size': 5,
    'use_embedding': True,
    'miss_linear_dim': 50
}

model = SimpleLSTM(config)
optimizer = model.optimizer

# Assuming 'model' is your trained model instance
model.save_model(file_path='models/model.pth')

# Assuming the saved model file is 'models/model.pth'
model_file_path = 'models/model.pth'

# Specify the device to load the model onto
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
loaded_model = BaseModel.load_model(SimpleLSTM, model_file_path)
# Now `loaded_model` is an instance of `SimpleLSTM` with the state and config loaded

model = loaded_model

In [6]:
from scr.dataset import ProcessedHangmanDataset

# # Load the dataset
processed_dataset = ProcessedHangmanDataset(pkls_dir, \
    char_frequency, max_word_length)

In [7]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader


# Convert PyTorch dataset to a list for train_test_split
dataset_list = [processed_dataset[i] for i in range(len(processed_dataset))]

# Perform an 80%-20% train-test split
train_data, val_data = train_test_split(dataset_list, \
    test_size=0.20, random_state=42)

del dataset_list

In [8]:
import random
from scr.model_training import train_on_data_loader


batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, \
    collate_fn=processed_dataset.custom_collate_fn)

optimizer = model.optimizer

# average_loss, average_miss_penalty \
#     = train_on_data_loader(model, train_loader, device, optimizer)


# # Define a probability threshold for inspecting a batch
# inspect_probability = 0.01  # Adjust this as needed

# for batch in train_loader:
#     if batch[0] is None:
#         continue  # Skip empty batches

#     # Randomly decide whether to inspect this batch
#     if random.random() < inspect_probability:
#         game_states_batch, lengths_batch, missed_chars_batch, labels_batch = batch
#         average_loss, average_miss_penalty = train_on_data_loader(model, )
#         print("Inspected Batch Shapes:")
#         print("Game States Batch Shape:", game_states_batch.shape)
#         print("Lengths Batch Shape:", lengths_batch.shape)
#         print("Missed Chars Batch Shape:", missed_chars_batch.shape)
#         print("Labels Batch Shape:", labels_batch.shape)
#         break

# average_loss, average_miss_penalty

In [9]:
from scr.model_training import validate_hangman

val_loader = processed_dataset.create_val_loader(val_data)


character_level, game_simulation  = validate_hangman(model, \
    val_loader, device, max_word_length, device)

# avg_loss, avg_miss_penalty, \
#     char_accuracy, word_accuracy, word_statistics

character_level, game_simulation

sweeteners
masked word:  ['sweeteners']


AttributeError: 'torch.device' object has no attribute 'get'

In [ ]:
len(val_loader)

In [ ]:
STOP

In [ ]:
batch_features_tensor.shape

In [ ]:
len(val_loader)

In [ ]:
batch_features_tensor.shape

In [ ]:
STOP

In [ ]:
STOP

In [ ]:
def train_epoch(model, data_loader, optimizer, device):
    model.train()  # Set the model to training mode
    total_loss = 0
    total_miss_penalty = 0

    for batch in data_loader:
        if batch[0] is None:
            continue  # Skip empty batches

        game_states_batch, lengths_batch, missed_chars_batch, labels_batch = batch
        game_states_batch, lengths_batch, missed_chars_batch = \
            game_states_batch.to(device), lengths_batch, missed_chars_batch.to(device)

        optimizer.zero_grad()

        outputs = model(game_states_batch, lengths_batch, missed_chars_batch)
        model_output_shape = outputs.shape
        reshaped_labels = pad_and_reshape_labels(labels_batch, model_output_shape).to(device)

        loss, miss_penalty = model.calculate_loss(outputs, reshaped_labels, \
                                                  lengths_batch, missed_chars_batch, 27)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_miss_penalty += miss_penalty.item()  # Accumulate miss penalty

    avg_loss = total_loss / len(data_loader)
    avg_miss_penalty = total_miss_penalty / len(data_loader)
    return avg_loss, avg_miss_penalty  # Return average loss and miss penalty



def validate_epoch(model, data_loader, device):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    total_miss_penalty = 0

    with torch.no_grad():
        for batch in data_loader:
            if batch[0] is None:
                continue  # Skip empty batches


            game_states_batch, lengths_batch, missed_chars_batch, labels_batch = batch
            game_states_batch, lengths_batch, missed_chars_batch = \
                game_states_batch.to(device), lengths_batch, missed_chars_batch.to(device)

            outputs = model(game_states_batch, lengths_batch, missed_chars_batch)
            model_output_shape = outputs.shape
            reshaped_labels = pad_and_reshape_labels(labels_batch, model_output_shape).to(device)

            loss, miss_penalty = model.calculate_loss(outputs, reshaped_labels, \
                                                      lengths_batch, missed_chars_batch, 27)
            total_loss += loss.item()

            total_miss_penalty += miss_penalty.item()  # Accumulate miss penalty

    avg_loss = total_loss / len(data_loader)
    avg_miss_penalty = total_miss_penalty / len(data_loader)
    return avg_loss, avg_miss_penalty  # 

# for epoch in range(num_epochs):
#     train_loss = train_epoch(model, train_loader, optimizer, device)
#     val_loss = validate_epoch(model, val_loader, device)
    
#     print(f"Epoch {epoch}: Training Loss: {train_loss}, Validation Loss: {val_loss}")
#     # You can add code to save model checkpoints if needed

In [ ]:
from sklearn.model_selection import KFold
import torch
from torch.utils.data import DataLoader, Subset

def train_model(model, train_data, val_data, num_epochs, optimizer, scheduler, device):
    train_loader = DataLoader(train_data, batch_size=batch_size, \
        collate_fn=custom_collate_fn)
    val_loader = DataLoader(val_data, batch_size=batch_size, \
        collate_fn=custom_collate_fn)

    for epoch in range(num_epochs):
        train_loss, train_miss_penalty = train_epoch(model, train_loader, \
            optimizer, device)
        val_loss, val_miss_penalty = validate_epoch(model, \
            val_loader, device)

        scheduler.step(val_loss)  # Adjust LR based on validation loss

        print(f"Epoch {epoch}: Training Loss: {train_loss}, Miss Penalty: \
            {train_miss_penalty}, Validation Loss: {val_loss}, Validation Miss Penalty: {val_miss_penalty}")
        # Save model checkpoints if needed

def k_fold_cross_validate(model, dataset, k, num_epochs, optimizer, scheduler_class, device):
    kfold = KFold(n_splits=k, shuffle=True, random_state=42)
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        print(f"Fold {fold}")
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        # Initialize a new model for each fold
        model = SimpleLSTM(config)
        model.to(device)
        optimizer = model.optimizer
        scheduler = scheduler_class(optimizer)

        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)
        train_model(model, train_subset, val_subset, num_epochs, optimizer, scheduler, device)


        train_model(model, train_subset, val_subset, num_epochs, optimizer, scheduler, device)

# Usage example
num_epochs = 10
batch_size = 64
scheduler_class = torch.optim.lr_scheduler.ReduceLROnPlateau  # Example scheduler class
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

k_fold_cross_validate(model, processed_dataset, \
    5, num_epochs, optimizer, scheduler_class, device)

In [ ]:
STOP

In [ ]:
from scr.dataset import ProcessedHangmanDataset
from scr.custom_sampler import PerformanceBasedSampler


print(f"Total number of data points in the dataset: {len(processed_dataset)}")

# Initialize the sampler
sampler = PerformanceBasedSampler(
    processed_dataset, 
    performance_metrics,
    max_word_length=max_word_length
)

print(f"Sampler created with {len(sampler)} indices.")

In [ ]:
sample_indices = list(sampler)[:10]  # Get the first 10 sampled indices
print("Sampled indices:", sample_indices)

for idx in sample_indices:
    data_point = processed_dataset[idx]
    print(f"Data at index {idx}: {data_point}")

In [ ]:
from scr.utils import print_scenarios

def process_pkl_files(base_dir):
    pkl_list = []

    # Iterate over all batch directories
    for batch_dir in sorted(base_dir.iterdir(), key=lambda x: int(x.name) if x.name.isdigit() else float('inf')):
        if batch_dir.is_dir():
            # List all .pkl files in the current batch directory
            pkl_files = list(batch_dir.glob("*.pkl"))

            for pkl_file in pkl_files:
                try:
                    with open(pkl_file, 'rb') as file:
                        game_data = pickle.load(file)
                except IOError as e:
                    print(f"Error reading file {pkl_file}: {e}")
                    continue

                # Processing each pickle file
                pkl_list.extend(process_pkl_file(pkl_file, game_data))

    return pkl_list

def process_pkl_file(pkl_file, game_data):
    file_scenarios = []
    for data in game_data:
        game_won, guesses = data
        word, initial_state, difficulty, outcome = extract_info_from_filename(pkl_file)
        
        # Create a scenario dictionary for each data tuple
        scenario = {
            'word': word,
            'difficulty': difficulty,
            'outcome': outcome,
            'data': (game_won, guesses)
        }
        file_scenarios.append((pkl_file, scenario))  # Add scenario to the list

    return file_scenarios

def extract_info_from_filename(pkl_file):
    parts = pkl_file.stem.split('_from_')
    word_and_state = parts[0].split('_')
    word = '_'.join(word_and_state[:-1])
    initial_state = word_and_state[-1]
    difficulty, outcome = parts[1].split('_')[-2:]
    return word, initial_state, difficulty, outcome

# def print_scenarios(scenarios):
#     # Assuming this function is defined elsewhere
#     pass

# Process all pickle files
pkl_list = process_pkl_files(base_dataset_dir)

# Accessing an individual pickle file's content by index
index_to_access = 0  # Change this index to access different files
if index_to_access < len(pkl_list):
    file_path, scenario = pkl_list[index_to_access]
    print(f"Contents of {file_path}:")
    print_scenarios([scenario])  # Wrap scenario in a list for the function
else:
    print(f"No pickle file at index {index_to_access}")

No pickle file at index 0



In [ ]:
pkl_list = []

# Iterate over all batch directories
for batch_dir in sorted(base_dataset_dir.iterdir(), \
    key=lambda x: int(x.name) if x.name.isdigit() else float('inf')):
    if batch_dir.is_dir():
        # List all .pkl files in the current batch directory
        pkl_files = list(batch_dir.glob("*.pkl"))

        for pkl_file in pkl_files:
            with open(pkl_file, 'rb') as file:
                game_data = pickle.load(file)
                # Extract information from file name
                parts = pkl_file.stem.split('_from_')
                word_and_statet = parts[0].split('_')
                word = '_'.join(word_and_state[:-1])
                initial_state = word_and_state[-1]
                difficulty, outcome = parts[1].split('_')[-2:]

                # Assuming game_data is a list of tuples (game_won, guesses)
                for data in game_data:
                    game_won, guesses = data
                    # Create a scenario dictionary for each data tuple
                    scenario = {
                        'word': word,
                        'difficulty': difficulty,
                        'outcome': outcome,
                        'data': (game_won, guesses)
                    }
                    pkl_list.append((pkl_file, scenario))  # Add scenario to the list

# Accessing an individual pickle file's content by index
index_to_access = 0  # Change this index to access different files
if index_to_access < len(pkl_list):
    file_path, scenario = pkl_list[index_to_access]
    print(f"Contents of {file_path}:")
    print_scenarios([scenario])  # Wrap scenario in a list for the function
else:
    print(f"No pickle file at index {index_to_access}")